In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
 
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:    

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
web = Website("https://www.linkedin.com/jobs/working-student-jobs-germany/")

In [6]:
link_system_prompt = "You are provided with a list of links for a job on a webpage. \
You are able to decide which of the links would be most relevant to the field of Data engineering or Artificial Intelligence, \
such as links for working student jobs in those fields.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "Artificial Intelligence", "url": "https://full.url/goes/here/about"},
        {"type": "Data Engineering": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the job website of {website.url} - "
    
    user_prompt += "please decide which of these are relevant web links that are visible on the company website for a job in the field of Data Engineering or Artificial Intelligence\n"
    user_prompt += "please remove the links which are of no use such as privacy and agreements.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [9]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [22]:
system_prompt = "You are an assistant in analyzes the contents of several relevant job openings from a job website \
and creates a short description about each job, the company offering the job and requirements for the job for prospective employees. Respond in markdown.\
Include details of jobs and requirements if you have the information."

In [23]:
def get_brochure_user_prompt(url):
    user_prompt = f"You are looking at a website entailing multiple job openings\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a description of each job and their relevant content such as requirements and other information pertaining to the job in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
def create_jobs(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(url)}
          ],
    )
    result = response.choices[0].message.content
    return result

In [51]:
create_jobs("https://www.linkedin.com/jobs/working-student-jobs-germany/")

Found links: {'links': [{'type': 'Data Engineering', 'url': 'https://de.linkedin.com/jobs/view/data-analytics-working-student-at-beat81-4131023280?position=21&pageNum=0&refId=lONQAnJxkKO%2F6yPdGGnnyw%3D%3D&trackingId=6vcC95RNMzSoyUkjyOHmcw%3D%3D'}, {'type': 'Artificial Intelligence', 'url': 'https://de.linkedin.com/jobs/view/machine-learning-working-student-hybrid-at-dida-4015219243?position=15&pageNum=0&refId=lONQAnJxkKO%2F6yPdGGnnyw%3D%3D&trackingId=I9hJH93N62xKdpSYWQClWA%3D%3D'}, {'type': 'Data Engineering', 'url': 'https://de.linkedin.com/jobs/view/working-student-data-science-f-m-d-at-ocell-4126026022?position=26&pageNum=0&refId=lONQAnJxkKO%2F6yPdGGnnyw%3D%3D&trackingId=1mlEmxEef3p%2FV7M6UPAfMQ%3D%3D'}, {'type': 'Artificial Intelligence', 'url': 'https://de.linkedin.com/jobs/view/working-student-internship-at-software-defined-automation-3837962677?position=53&pageNum=0&refId=lONQAnJxkKO%2F6yPdGGnnyw%3D%3D&trackingId=aW7520pmXgVkfKLJV4PfuQ%3D%3D'}]}


# Job Overview: Working Student Opportunities in Germany

## Company Description
In Germany, over 25,000 working student positions are available across various industries, providing valuable experiences while completing academic studies. Key players in this landscape include technology firms, startups, financial services, and diversified conglomerates such as NETZSCH Group, VÆRIDION, and TikTok. These roles cater primarily to students seeking part-time opportunities that align with their fields of study, helping them to gain relevant skills and industry experience.

---

## Job Listings

### 1. Working Student - Software Development (m/w/d)  
**Company:** perto  
**Location:** Berlin, Germany  
**Date Posted:** 19 hours ago  

**Job Description:**  
As a Working Student in Software Development, you will assist in developing and maintaining software applications, contributing to ongoing projects, and learning from experienced developers.

**Requirements:**  
- Enrolled in a Computer Science or related degree  
- Familiarity with programming languages such as Java, Python, or JavaScript  
- Problem-solving orientation and the ability to work in a team  
- Availability to work 15-20 hours per week

---

### 2. TikTok Product Marketing Working Student  
**Company:** TikTok  
**Location:** Berlin/Paris  
**Start Date:** Immediate, 2025  
**Status:** Actively Hiring  

**Job Description:**  
Join the Product Marketing team to support marketing initiatives and campaigns, focusing on user engagement strategies across various platforms.

**Requirements:**  
- Background in Marketing, Communications, or Business  
- Excellent communication skills in both written and oral formats  
- Experience with social media analytics tools is a plus  
- Creative mindset with strong analytical abilities

---

### 3. Data Analytics Working Student  
**Company:** BEAT81  
**Location:** Berlin, Germany  
**Date Posted:** 1 day ago  

**Job Description:**  
You will support the data analytics team in extracting and analyzing data, helping to drive decision-making processes.

**Requirements:**  
- Pursuing a degree in Data Science, Statistics, or similar  
- Proficient in SQL and data visualization tools  
- Strong analytical and problem-solving skills  
- Ability to work part-time during the week

---

### 4. Working Student Finance (all genders)  
**Company:** Q ENERGY  
**Location:** Berlin, Germany  
**Date Posted:** 11 hours ago  

**Job Description:**  
Assist the finance team in preparing reports, analyzing financial data, and supporting budgeting processes.

**Requirements:**  
- Enrolled in a Finance, Economics, or Accounting program  
- Detail-oriented with strong numerical skills  
- Basic knowledge of financial software is advantageous  
- Ability to commit to at least 15 hours a week

---

### 5. Working Student IT Support (m/w/d)  
**Company:** roadsurfer  
**Location:** Greater Munich Metropolitan Area  
**Date Posted:** 1 week ago  

**Job Description:**  
Provide technical support to users, assist in troubleshooting issues, and maintain IT equipment.

**Requirements:**  
- Studying Computer Science or a related field  
- Strong problem-solving skills and customer service orientation  
- Familiarity with IT support tools and systems  
- Flexible working hours are available

---

## Conclusion
These opportunities cater to students looking to integrate their academic learning with practical experience. Whether in software development, finance, marketing, or IT support, roles vary widely, ensuring that students have ample options to align with their career interests. Students can navigate this landscape effectively to build their resumes while pursuing academic goals.

In [13]:
import gradio as gr

In [24]:
view = gr.Interface(
    fn=create_jobs,
    inputs=[
        gr.Textbox(label="Landing page URL including http:// or https://")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Found links: {'links': [{'type': 'Data Engineering', 'url': 'https://de.linkedin.com/jobs/view/tiktok-product-marketing-working-student-berlin-paris-immediate-start-2025-at-tiktok-4125293886?position=1&pageNum=0&refId=goqYsHfchh%2BukgIF3AgzDw%3D%3D&trackingId=%2Bcuhi4P9uGWJoY3qE2rtTA%3D%3D'}, {'type': 'Artificial Intelligence', 'url': 'https://de.linkedin.com/jobs/view/semester-final-thesis-at-v%C3%A6ridion%C2%A0-all-genders-at-v%C3%A6ridion-4129496453?position=2&pageNum=0&refId=goqYsHfchh%2BukgIF3AgzDw%3D%3D&trackingId=1TIZmcIwRryGePwabgqUXQ%3D%3D'}, {'type': 'Data Engineering', 'url': 'https://de.linkedin.com/jobs/view/working-student-internship-ventures-team-at-campus-founders-3995387982?position=3&pageNum=0&refId=goqYsHfchh%2BukgIF3AgzDw%3D%3D&trackingId=ICB6UGJMDNCRE4sIWabvYg%3D%3D'}, {'type': 'Data Engineering', 'url': 'https://de.linkedin.com/jobs/view/working-student-internship-ventures-team-at-campus-founders-3995967547?position=4&pageNum=0&refId=goqYsHfchh%2BukgIF3AgzDw%3D%3D